<a href="https://colab.research.google.com/github/twillixa/HEC/blob/main/PPS/Week2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pulp
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpInteger, LpContinuous, value, LpStatus
import numpy as np
import pandas as pd

In [4]:
def solve_lp_basicaggregate(
    demand,
    production_per_worker,
    starting_inventory,
    desired_ending_inventory,
    initial_workers,
    wage_per_worker,
    hire_cost,
    layoff_cost,
    inventory_cost_per_unit
):
    months = len(demand)
    model = LpProblem("Aggregate_Production_Planning", LpMinimize)

    W = LpVariable.dicts("W", range(months), lowBound=0, cat=LpInteger)
    H = LpVariable.dicts("H", range(months), lowBound=0, cat=LpInteger)
    L = LpVariable.dicts("L", range(months), lowBound=0, cat=LpInteger)
    P = LpVariable.dicts("P", range(months), lowBound=0, cat=LpContinuous)
    I = LpVariable.dicts("I", range(months), lowBound=0, cat=LpContinuous)

    model += lpSum(
        W[t] * wage_per_worker +
        H[t] * hire_cost +
        L[t] * layoff_cost +
        I[t] * inventory_cost_per_unit
        for t in range(months)
    )

    for t in range(months):
        model += P[t] == W[t] * production_per_worker

        if t == 0:
            model += W[t] == initial_workers + H[t] - L[t]
            model += I[t] == starting_inventory + P[t] - demand[t]
        else:
            model += W[t] == W[t-1] + H[t] - L[t]
            model += I[t] == I[t-1] + P[t] - demand[t]

    model += I[t] >= desired_ending_inventory

    model.solve()

    status = LpStatus[model.status]
    obj = value(model.objective)

    df = pd.DataFrame({
        "Month": np.arange(1, months+1),
        "Workers": [W[t].varValue for t in range(months)],
        "Hired": [H[t].varValue for t in range(months)],
        "Laid off": [L[t].varValue for t in range(months)],
        "Produced": [P[t].varValue for t in range(months)],
        "Demand": demand,
        "End Inventory": [I[t].varValue for t in range(months)],
    })

    return df, obj, status


df, total_cost, status = solve_lp_basicaggregate(
demand = [600, 700, 400, 500, 300],
production_per_worker = 100,
starting_inventory = 200,
desired_ending_inventory = 200,
initial_workers = 10,
wage_per_worker = 4,
hire_cost = 1.5,
layoff_cost = 4,
inventory_cost_per_unit = 0.05
)